# パッケージインストール

In [1]:
!git clone https://github.com/sczhou/ProPainter.git
%cd ProPainter
!pip install -q -r requirements.txt

Cloning into 'ProPainter'...
remote: Enumerating objects: 608, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 608 (delta 65), reused 50 (delta 50), pack-reused 498 (from 1)
Receiving objects: 100% (608/608), 55.20 MiB | 56.02 MiB/s, done.
Resolving deltas: 100% (98/98), done.
/content/ProPainter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 19.3 MB/s eta 0:00:00


# 重みダウンロード

In [2]:
!wget https://github.com/sczhou/ProPainter/releases/download/v0.1.0/i3d_rgb_imagenet.pt -P weights -q
!wget https://github.com/sczhou/ProPainter/releases/download/v0.1.0/ProPainter.pth -P weights -q
!wget https://github.com/sczhou/ProPainter/releases/download/v0.1.0/raft-things.pth -P weights -q
!wget https://github.com/sczhou/ProPainter/releases/download/v0.1.0/recurrent_flow_completion.pth -P weights -q

# サンプル動画ダウンロード

NHKクリエイティブライブラリーの「[ドバイ（３）道路渋滞 アップ](https://www2.nhk.or.jp/archives/movies/?id=D0002050330_00000)」の後半部分を切り出して使用しています

In [3]:
import gdown

gdown.download(
    'https://drive.google.com/uc?id=11LnrwjFAs6NQc4AU9ejrqpJ9eg85Yl_H',
    'sample.mp4',
    quiet=False,
)
gdown.download(
    'https://drive.google.com/uc?id=1BfBOWp0u8fgTI8upFW-hK83JGMW40fbo',
    'mask.png',
    quiet=False,
)

Downloading...
From: https://drive.google.com/uc?id=11LnrwjFAs6NQc4AU9ejrqpJ9eg85Yl_H
To: /content/ProPainter/sample.mp4
100%|██████████| 1.23M/1.23M [00:00<00:00, 140MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BfBOWp0u8fgTI8upFW-hK83JGMW40fbo
To: /content/ProPainter/mask.png
100%|██████████| 2.65k/2.65k [00:00<00:00, 2.55MB/s]


'mask.png'

In [4]:
import os

# 保存ディレクトリの作成
frame_dir = 'inputs/test/frame'
mask_dir = 'inputs/test/mask'
os.makedirs(frame_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

In [5]:
import cv2
from tqdm import tqdm

# 動画読み込み
cap = cv2.VideoCapture('sample.mp4')
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# マスク画像読み込み
mask_image = cv2.imread("mask.png")

# 画像化
frame_count = 0
with tqdm(total=total_frames, desc="Processing Video") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 元のサイズのままだとA100でもメモリオーバーのためリサイズ
        frame = cv2.resize(frame, (800, 464))

        height, width, _ = frame.shape

        # フレームとマスク画像を保存
        frame_name = f"{frame_count:05d}.png"
        cv2.imwrite(os.path.join(frame_dir, frame_name), frame)
        cv2.imwrite(os.path.join(mask_dir, frame_name), mask_image)

        # フレームカウンターをインクリメント
        frame_count += 1
        pbar.update(1)

    cap.release()

Processing Video: 100%|██████████| 698/698 [00:19<00:00, 36.10it/s]


In [6]:
print(width, height)

800 464


In [8]:
%%time

# 画像サイズが大きいためA100必須
!python inference_propainter.py \
    --width 800 --height 464 \
    --video inputs/test/frame \
    --mask inputs/test/mask

Pretrained flow completion model has loaded...
Pretrained ProPainter has loaded...
Network [InpaintGenerator] was created. Total number of parameters: 39.4 million. To see the architecture, do print(network).

Processing: frame [698 frames]...
100% 140/140 [02:00<00:00,  1.16it/s]

All results are saved in results/frame
CPU times: user 1.37 s, sys: 203 ms, total: 1.57 s
Wall time: 4min 9s
